# Pillar 5 · Stakeholder Engagement

Coordinate cross-functional stakeholders to oversee Azure AI agent governance.

## Learning Objectives

- Define governance roles and engagement cadence
- Share agent health insights with business stakeholders
- Capture feedback loops that improve agent safety and performance

## Engagement Plan

1. **Stakeholder Mapping**: Document accountable, responsible, consulted, and informed parties for each agent lifecycle stage.
2. **Communication Artifacts**: Use dashboards populated from telemetry in `04_deployment_monitoring.ipynb` to brief executives and risk teams.
3. **Feedback Capture**: Write interaction summaries back to Cosmos DB via `AgentDB.update_agent` to close the loop on stakeholder feedback.
4. **Governance Reviews**: Schedule recurring sessions to evaluate policy effectiveness and prioritize remediation items.

## Collaboration Checklist

- Roles and responsibilities documented
- Reporting templates aligned with stakeholder needs
- Feedback items tracked through resolution

## Configure PATH for Azure CLI

Ensure the Azure CLI is accessible in the notebook kernel's PATH.

In [ ]:
import os
import shutil

# Replace with the directory you want to add
new_path_entry = "/opt/homebrew/bin"
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

In [ ]:
# Retrieve a transcript and enrich it with governance tags
import os
import sys
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))
from agent_utils import AgentManager

# Load environment variables
load_dotenv("../.env")


print("✅ Imports successful")

endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
if not endpoint:
    raise ValueError(
        "Set AZURE_AI_PROJECT_ENDPOINT in .env.local before running this notebook.")

project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

manager = AgentManager(project_client)

In [ ]:
# Build a RACI matrix for agent governance stages
import pandas as pd

raci_rows = [
    {"Stage": "Design", "Responsible": "Product Owner", "Accountable": "AI Governance Lead", "Consulted": "Legal", "Informed": "Security"},
    {"Stage": "Evaluation", "Responsible": "Responsible AI Team", "Accountable": "AI Governance Lead", "Consulted": "Data Science", "Informed": "Compliance"},
    {"Stage": "Deployment", "Responsible": "MLOps", "Accountable": "Platform Lead", "Consulted": "Security", "Informed": "Business Owner"},
    {"Stage": "Monitoring", "Responsible": "MLOps", "Accountable": "AI Governance Lead", "Consulted": "Support", "Informed": "Stakeholder Council"},
    {"Stage": "Retirement", "Responsible": "Product Owner", "Accountable": "Platform Lead", "Consulted": "Legal", "Informed": "All Users"}
 ]

raci_matrix = pd.DataFrame(raci_rows)
raci_matrix

In [ ]:
# Record stakeholder feedback in Cosmos DB
import os
from datetime import datetime
from agent_db import AgentDB

db = AgentDB() 

agent_id = "asst_CncymMdTqov5hRCQQJrvLwhX"  # Replace with actual agent ID

agent_record = db.get_agent(azure_agent_id=agent_id) or {}
history = agent_record.get("governanceFeedback", [])

feedback_entry = {
    "timestamp": datetime.utcnow().isoformat() + "Z",
    "stakeholder": "Risk Committee",
    "summary": "Requested clearer escalation playbook for critical incidents.",
    "actionItems": ["Draft escalation swimlane", "Schedule tabletop exercise"],
    "status": "open"
}

history.append(feedback_entry)

db.update_agent(
    azure_agent_id=agent_id,
    governanceFeedback=history
)
print("Feedback logged to governance record.")

## Next Notebook

Proceed to `07_technical_safeguards.ipynb` to finalize technical guardrails.